# GONZAG cloud example

In [1]:
import sys
from os import getenv
import warnings
warnings.filterwarnings("ignore")

In [2]:
from gonzag.config import *
from gonzag.io import *
from gonzag.utils import *
from gonzag.mod2sat import *
from gonzag.bilin_mapping import *
from gonzag.spectralysis import *


## Data

In [3]:
import xarray as xr
osn_url = 'https://ncsa.osn.xsede.org/Pangeo/pangeo-forge/swot_adac/eNATL60_surface_region_1'
dsmod = xr.open_zarr(osn_url, consolidated=True)

In [4]:
dsmod

<xarray.Dataset>
Dimensions:        (time_counter: 2136, x: 611, y: 763)
Coordinates:
    depth          (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e1v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2f            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2t            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2u            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    e2v            (y, x) float64 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lat            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    lon            (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lat        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    nav_lon        (y, x) float32 dask.array<chunksize=(763, 611), meta=np.ndarray>
    time_centered  (time_counter) datetime64[ns] dask.array<chunksize=(72,), meta=np.ndarray>
  * time_counter   (time_counter) datetime64[ns] 2010-02-01T00:30:00 ... 2010...
Dimensions without coordinates: x, y
Data variables:
    fmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    qt_oce         (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    somecrty       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sometauy       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosaline       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sossheig       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sosstsst       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sowaflup       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozocrtx       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    sozotaux       (time_counter, y, x) float32 dask.array<chunksize=(72, 763, 611), meta=np.ndarray>
    tmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    umask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>
    vmask          (y, x) int8 dask.array<chunksize=(763, 611), meta=np.ndarray>

In [5]:
name_ssh_mod = 'sossheig'
name_lsm_mod = 'tmask' 
l_griddist = False

In [6]:
from intake import open_catalog
cat = open_catalog("https://raw.githubusercontent.com/pangeo-data/pangeo-datastore/master/intake-catalogs/ocean/altimetry.yaml")
dssat  = cat["al"].to_dask()

In [7]:
dssat

<xarray.Dataset>
Dimensions:         (time: 34340107)
Coordinates:
    latitude        (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    longitude       (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
  * time            (time) datetime64[ns] 2013-03-14T05:44:50.445185024 ... 2...
Data variables:
    cycle           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
    dac             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    lwe             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    mdt             (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    ocean_tide      (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_filtered    (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    sla_unfiltered  (time) float64 dask.array<chunksize=(34340107,), meta=np.ndarray>
    track           (time) int16 dask.array<chunksize=(34340107,), meta=np.ndarray>
Attributes:
    Conventions:               CF-1.6
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    cdm_data_type:             Swath
    comment:                   Sea surface height measured by altimeters refe...
    contact:                   servicedesk.cmems@mercator-ocean.eu
    creator_email:             servicedesk.cmems@mercator-ocean.eu
    creator_name:              CMEMS - Sea Level Thematic Assembly Center
    creator_url:               http://marine.copernicus.eu
    history:                   2019-02-20T19:39:52Z: Creation
    institution:               CLS, CNES
    keywords:                  Oceans > Ocean Topography > Sea Surface Height
    keywords_vocabulary:       NetCDF COARDS Climate and Forecast Standard Names
    license:                   http://marine.copernicus.eu/web/27-service-com...
    platform:                  Altika
    processing_level:          L3
    product_version:           2019
    project:                   COPERNICUS MARINE ENVIRONMENT MONITORING SERVI...
    references:                http://marine.copernicus.eu
    software_version:          6.2_DUACS_DT2018_baseline
    source:                    Altika measurements
    ssalto_duacs_comment:      The reference mission used for the altimeter i...
    standard_name_vocabulary:  NetCDF Climate and Forecast (CF) Metadata Conv...
    summary:                   SSALTO/DUACS Delayed-Time Level-3 sea surface ...
    title:                     DT Altika Global Ocean Along track SSALTO/DUAC...

In [8]:
name_ssh_sat='sla_unfiltered'

## Time overlap between model and satellite ?
Assume that model is less than 1 year, satellite has at least one whole year, we take the overlapping period without considering which year is this

In [9]:
    (date1,date2), (Nts,Ntm) = GetTimeOverlap( dssat , dsmod )
    print(' *** Time overlap between model and satellite in UNIX epoch time: it1, it2',date1,'--',date2)


 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time"
   => first and last date:  2013-03-14T05:44:50.445185024 -- 2015-03-31T22:41:36.909480960
 *** [GetTimeInfo()] Getting calendar/time info in dataset ...
   => time/record dimension is "time_counter"
   => first and last date:  2010-02-01T00:30:00.000000000 -- 2010-04-30T23:30:00.000000000

 *** Earliest/latest dates:
   => for satellite data: 2013-03-14T05:44:50.445185024 2015-03-31T22:41:36.909480960 
   => for model     data: 2010-02-01T00:30:00.000000000 2010-04-30T23:30:00.000000000 

 *** Time overlap between model and satellite in UNIX epoch time: it1, it2 2014-02-01 -- 2014-04-30


## Create object `ModelGrid` containing all the model (aka _source_) 2D+T domain grid info

In [10]:
  
    ModelGrid = ModGrid( dsmod, date1 , date2 , dsmod, name_lsm_mod, distorded_grid=False )


 *** [GetModelCoor()] Read model latitude (variable is "lat", with 2 dimensions! (763, 611) 

 *** [GetModelCoor()] Read model longitude (variable is "lon", with 2 dimensions! (763, 611) 


 *** what we use to define model land-sea mask:
    => "tmask" in dataset 

 *** [GridResolution()] Based on the longitude array, the model resolution ~=  0.016657295  degrees 

 *** Skipping computation of angle distortion of the model grid! ("-D" option not invoked)...

 *** About model gridded (source) domain:
     * shape =  (763, 611)
     * horizontal resolution:  0.016657295  degrees or  1.8507920147292316  km
     * Is this a global domain w.r.t longitude:  False
       ==> this is a regional domain
       ==> working in the [0:360] frame...
     * lon_min, lon_max =  281.99 292.17
     * lat_min, lat_max =  30.01 40.01
     * should we pay attention to possible STRONG local distorsion in the grid:  False
     * number of time records of interest for the interpolation to come:  2136
       =

In [11]:
SatelliteTrack = SatTrack( dssat, date1, date2, Np=Nts, \
                              domain_bounds=ModelGrid.domain_bounds, l_0_360=ModelGrid.l360 )

 *** [SatTrack()] Analyzing the time vector in dataset ...
 *** [GetSatCoor()] reading "latitude" in dataset ...
   => 4150072 records read...

 *** [GetSatCoor()] reading "longitude" in dataset ...
   => 4150072 records read...


 *** About satellite track (target) domain:
     * number of time records of interest for the interpolation to come:  10236
       ==> time record indices: 14883510 to 19033581, included

       separated in 87 tracks


In [13]:
MG= ModelGrid
ST= SatelliteTrack

In [14]:
from gonzag.io   import GetModel2DVar, GetSatSSH
from gonzag.bilin_mapping import NearestPoint
from gonzag.bilin_mapping import IDSourceMesh
from gonzag.bilin_mapping import WeightBL
import pandas as pd


(Nj,Ni) = MG.shape

d_found_km = rfactor*MG.HResDeg*deg2km
print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

# Size of the search zoom box:
np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

Nt = ST.size ; # number of satellit observation point to work with here...

if_talk = Nt//nb_talk

Yt=ST.lat.values
Xt=ST.lon.values
Ys=MG.lat.values
Xs=MG.lon.values
src_grid_local_angle=MG.xangle
k_ew_per=MG.EWPer
rd_found_km=d_found_km
np_box_r=np_box_radius
freq_talk=if_talk

sangle = src_grid_local_angle
kewp   = k_ew_per
rfound = rd_found_km
nprad  = np_box_r
(Nj,Ni) = Ys.shape
        
NP = nmp.zeros((1,2)  , dtype=nmp.int64) ; # nearest point
SM = nmp.zeros((1,4,2), dtype=nmp.int64) ; # source mesh
WB = nmp.zeros((1,4))                    ; # weights
vssh_m_np = nmp.zeros(Nt) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
vssh_m_bl = nmp.zeros(Nt) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
vdistance = nmp.zeros(Nt)

[jinit,iinit] = [nprad,nprad] ; # stupid first guess here...


 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940110469238  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...



In [15]:
ST.index_tracks

[[0, 167],
 [503, 651],
 [651, 824],
 [1137, 1216],
 [1216, 1327],
 [1327, 1464],
 [1469, 1540],
 [1540, 1706],
 [2050, 2221],
 [2221, 2394],
 [2545, 2717],
 [2866, 3039],
 [3363, 3439],
 [3439, 3591],
 [3591, 3760],
 [3761, 3791],
 [3791, 3964],
 [4120, 4292],
 [4620, 4769],
 [4769, 4942],
 [5249, 5326],
 [5326, 5437],
 [5437, 5571],
 [5576, 5638],
 [5638, 5800],
 [6130, 6292],
 [6292, 6458],
 [6608, 6780],
 [6930, 7103],
 [7420, 7498],
 [7498, 7631],
 [7631, 7802],
 [7803, 7845],
 [7845, 8018],
 [8190, 8362],
 [8709, 8860],
 [8860, 9028],
 [9340, 9413],
 [9413, 9522],
 [9522, 9657],
 [9662, 9733],
 [9733, 9896],
 [168, 338],
 [339, 503],
 [825, 947],
 [948, 973],
 [974, 1026],
 [1027, 1137],
 [1465, 1469],
 [1707, 1879],
 [1880, 2050],
 [2395, 2545],
 [3040, 3126],
 [3127, 3193],
 [3194, 3289],
 [3965, 4120],
 [4293, 4465],
 [4466, 4620],
 [4943, 5062],
 [5063, 5088],
 [5089, 5138],
 [5139, 5249],
 [5572, 5576],
 [5801, 5961],
 [5962, 6130],
 [6459, 6608],
 [7104, 7192],
 [7193, 7262

In [18]:
ST.index_tracks[0][1]

167

In [21]:
print(np.arange(ST.index_tracks[0][0],ST.index_tracks[0][1]))

[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166]


In [26]:
ST.lat[ST.index_tracks[0][0]:ST.index_tracks[0][1]+1]

<xarray.Variable (time: 168)>
dask.array<getitem, shape=(168,), dtype=float64, chunksize=(168,), chunktype=numpy.ndarray>
Attributes:
    long_name:      Latitude of measurement
    standard_name:  latitude
    units:          degrees_north

In [25]:
ST.index_tracks[0][0]

0

In [27]:
        one_track=0
        jt1=ST.index_tracks[one_track][0]
        jt2=ST.index_tracks[one_track][1]

        from gonzag.io   import GetModel2DVar, GetSatSSH

        (Nj,Ni) = MG.shape

        d_found_km = rfactor*MG.HResDeg*deg2km
        print(' *** "found" distance criterion when searching for nearest point on model grid is ', d_found_km, ' km\n')

        # Size of the search zoom box:
        np_box_radius = SearchBoxSize( MG.HResDeg*deg2km, search_box_w_km )
        print(' *** Will use zoom boxes of width of '+str(2*np_box_radius+1)+' points for 1st attempts of nearest-point location...\n')

        Nt = jt2 - jt1 ; # number of satellit observation point to work with here...

        if_talk = Nt//nb_talk

        # The BIG GUY:
        startTime = time.time()

        BT = BilinTrack( ST.lat[jt1:jt2+1], ST.lon[jt1:jt2+1], MG.lat, MG.lon, src_grid_local_angle=MG.xangle, \
                         k_ew_per=MG.EWPer, rd_found_km=d_found_km, np_box_r=np_box_radius, freq_talk=if_talk )

        time_bl_mapping = time.time() - startTime

        print('     - Construction of the source-target bilinear mapping took: '+str(round(time_bl_mapping,0))+' s')


 *** "found" distance criterion when searching for nearest point on model grid is  1.3880940110469238  km

 *** Will use zoom boxes of width of 271 points for 1st attempts of nearest-point location...


 *** Finding nearest points on source (model) grid... (rd_found_km, np_box_r = 1.3880940110469238 135 )
      +++ Treated point: 16/168 
          ==> Sat. coordinates:     39.11 289.402
          ==> Model nearest point:  39.111 289.395  ( 692 451 )
      +++ Treated point: 32/168 
          ==> Sat. coordinates:     38.189 289.109
          ==> Model nearest point:  38.191 289.105  ( 617 432 )
      +++ Treated point: 48/168 
          ==> Sat. coordinates:     37.268 288.822
          ==> Model nearest point:  37.269 288.828  ( 543 414 )
      +++ Treated point: 64/168 
          ==> Sat. coordinates:     36.057 288.454
          ==> Model nearest point:  36.062 288.452  ( 448 390 )
      +++ Treated point: 80/168 
          ==> Sat. coordinates:     35.134 288.179
          ==> Mode

In [29]:
ST.time

<xarray.DataArray 'time' (time: 10236)>
array(['2014-02-02T23:14:13.207443968', '2014-02-02T23:14:14.187443968',
       '2014-02-02T23:14:15.167443968', ..., '2014-04-29T23:14:02.891097088',
       '2014-04-29T23:14:03.871097088', '2014-04-29T23:14:04.851097088'],
      dtype='datetime64[ns]')
Coordinates:
    latitude   (time) float64 dask.array<chunksize=(10236,), meta=np.ndarray>
    longitude  (time) float64 dask.array<chunksize=(10236,), meta=np.ndarray>
  * time       (time) datetime64[ns] 2014-02-02T23:14:13.207443968 ... 2014-0...
Attributes:
    axis:           T
    long_name:      Time of measurement
    standard_name:  time

In [30]:
Nt

167

In [32]:
jt1

0

In [36]:
            itt = ST.time[jt].values ; # unix time


In [34]:
itt

<xarray.DataArray 'time' ()>
array('2014-02-02T23:14:13.207443968', dtype='datetime64[ns]')
Coordinates:
    latitude   float64 dask.array<chunksize=(), meta=np.ndarray>
    longitude  float64 dask.array<chunksize=(), meta=np.ndarray>
    time       datetime64[ns] 2014-02-02T23:14:13.207443968
Attributes:
    axis:           T
    long_name:      Time of measurement
    standard_name:  time

In [42]:
BT.SM.shape

(168, 4, 2)

In [44]:
        vssh_m_np = nmp.zeros(Nt+1) ; vssh_m_np[:] = rmissval; # vector to store the model data interpolated in time and space (nearest-point) on the satellite track...
        vssh_m_bl = nmp.zeros(Nt+1) ; vssh_m_bl[:] = rmissval; # vector to store the model data interpolated in time and space (bilinear) on the satellite track...
        vdistance = nmp.zeros(Nt+1)

        # Time increment on the satellite time:
        ktm1   = 0   ; ktm2   = 0
        ktm1_o = -10 ; ktm2_o = -10

        print('\n *** Starting space-time interpolation of model data onto the '+str(Nt)+' selected track points...')

        startTime = time.time()

        for jt in range(jt1,jt2+1):

            # kt* : index for model
            # jt* : index for sat. track...

            itt = ST.time[jt].values ; # unix time
            year_sat=pd.to_datetime(itt).year
            date_model=pd.Series(MG.time.values)
            date_model_satyear=date_model.apply(lambda dt: dt.replace(year=year_sat))

            # Get surrounding records for model:
            kt = ktm1
            while not (date_model_satyear[kt]<=itt and date_model_satyear[kt+1]>itt): kt=kt+1
            ktm1 = kt ; ktm2 = kt+1

            if jt%if_talk==0:
                print('      jt = '+'%5.5i'%(jt)+' => satelite time = '+str(itt))
                if ivrb>0: print('   => surounding kt for model: ', ktm1, ktm2,    MG.time[ktm1].values,MG.time[ktm2].values )

            # If first time we have these ktm1 & ktm2, getting the two surrounding fields:
            if (ktm1>ktm1_o) and (ktm2>ktm2_o):
                if (ktm1_o == -10) or (ktm1 > ktm2_o):
                    if ivrb>0: print(' *** Reading '+name_ssh_mod+' in model dataset \n    => at ktm1=', ktm1)
                    Xm1 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm1 )
                else:
                    Xm1[:,:] = Xm2[:,:]
                #
                if ivrb>0: print(' *** Reading '+name_ssh_mod+' in in model dataset \n    => at ktm2=', ktm2)
                Xm2 = GetModel2DVar( MG.file, name_ssh_mod, kt=ktm2 )

                # slope only needs to be calculated when Xm2 and Xm1 have been updated:
                Xa = (Xm2 - Xm1) / float((MG.time[ktm2].values - MG.time[ktm1].values )/ np.timedelta64(1, 's'))

            # Linear interpolation of field at time itt:
            if ivrb>0 and jt%if_talk==0: print('   => Model data is interpolated at current time out of model records '+str(ktm1)+' & '+str(ktm2))
            Xm = Xm1[:,:] + Xa[:,:]*float((itt - date_model_satyear[ktm1])/ np.timedelta64(1, 's'))

            [ [j1,i1],[j2,i2],[j3,i3],[j4,i4] ] = BT.SM[jt-jt1,:,:]
            [w1, w2, w3, w4]                    = BT.WB[jt-jt1,:]

            Sm = MG.mask[j1,i1] + MG.mask[j2,i2] + MG.mask[j3,i3] + MG.mask[j4,i4]
            if Sm.values == 4:
                # All 4 model points are ocean point !

                vssh_m_np[jt-jt1] = Xm[j1,i1] ; # Nearest-point "interpolation"

                # Bilinear interpolation:
                Sw = nmp.sum([w1, w2, w3, w4])
                if abs(Sw-1.)> 0.001:
                    if ivrb>0: print('    FLAGGING MISSING VALUE at jt = '+str(jt)+' !!!')
                else:
                    vssh_m_bl[jt-jt1] = w1*Xm[j1,i1] + w2*Xm[j2,i2] + w3*Xm[j3,i3] + w4*Xm[j4,i4]

            ktm1_o = ktm1 ; ktm2_o = ktm2

        # end of loop on jt

        time_bl_interp = time.time() - startTime

        print(' *** Space-time interpolation done!\n')
        print('     - Interpolation of model data on the '+str(Nt)+' satellite points took: '+str(round(time_bl_interp,0))+' s \n')



 *** Starting space-time interpolation of model data onto the 167 selected track points...
      jt = 00000 => satelite time = 2014-02-02T23:14:13.207443968
 *** [GetModel2DVar()] Reading model "sossheig" at record kt=46 in dataset

 *** [GetModel2DVar()] Reading model "sossheig" at record kt=47 in dataset

      jt = 00016 => satelite time = 2014-02-02T23:14:28.887443968
      jt = 00032 => satelite time = 2014-02-02T23:14:44.567443968
      jt = 00048 => satelite time = 2014-02-02T23:15:00.247443968
      jt = 00064 => satelite time = 2014-02-02T23:15:20.827443968
      jt = 00080 => satelite time = 2014-02-02T23:15:36.507443968
      jt = 00096 => satelite time = 2014-02-02T23:15:52.187443968
      jt = 00112 => satelite time = 2014-02-02T23:16:07.867443968
      jt = 00128 => satelite time = 2014-02-02T23:16:23.547443968
      jt = 00144 => satelite time = 2014-02-02T23:16:39.227443968
      jt = 00160 => satelite time = 2014-02-02T23:16:54.907443968
 *** Space-time interpolation 